# Indexing

In [1]:
# Standard library
import os
import glob
import pickle
from pathlib import Path

# Third-party libraries
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer

/opt/anaconda3/envs/internship-2025/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load texts
chunks = []
for path in glob.glob("../data-source/text/*.txt"):
    with open(path, "r", encoding="utf-8") as f:
        text = f.read().strip()
    if text:
        chunks.append({"file_path": path, "text": text})
        print(f"{os.path.basename(path)}: loaded")
    else:
        print(f"{os.path.basename(path)}: skipped")

print(f"\nTotal valid chunks: {len(chunks)}")
if chunks:
    print(f"\nPreview: {chunks[0]['text'][:100]}")

# Build FAISS index
model = SentenceTransformer("intfloat/multilingual-e5-small")
vecs = model.encode([c["text"] for c in chunks], normalize_embeddings=True).astype("float32")
index = faiss.IndexFlatL2(vecs.shape[1])
index.add(vecs)
print(f"Registered in FAISS index: {index.ntotal} vectors")

# Save index and metadata
Path("../data-source/faiss").mkdir(parents=True, exist_ok=True)
faiss.write_index(index, "../data-source/faiss/faiss.index")
with open("../data-source/faiss/faiss_meta.pkl", "wb") as f:
    pickle.dump(chunks, f)



Barcelona.txt: loaded
Seoul.txt: loaded
Atlanta.txt: loaded
Sydney.txt: loaded

Total valid chunks: 4

Preview: 1992年バルセロナオリンピック（正式名称：第25回夏季オリンピック競技大会）は、スペインのバルセロナで開催された国際的なスポーツ大会で、以下のような特徴があります。

---

### 基本情報


Registered in FAISS index: 4 vectors


# Search

In [8]:
# Search
query = "オーストラリアで開催"
query_vec = model.encode([query], normalize_embeddings=True).astype("float32")
D, I = index.search(query_vec, k=4)

# Show results
df = pd.DataFrame([
    {"text": chunks[i]["text"][:100], "L2-distance": float(d)}
    for i, d in zip(I[0], D[0]) if i < len(chunks)
])
df

,text,L2-distance
0,【2000年 シドニーオリンピック（第27回夏季大会）】\n開催地：オーストラリア・シドニー...,0.334675
1,【1996年 アトランタオリンピック（第26回夏季大会）】\n開催地：アメリカ・アトランタ\...,0.344090
2,1992年バルセロナオリンピック（正式名称：第25回夏季オリンピック競技大会）は、スペインの...,0.345896
3,【1988年 ソウルオリンピック（第24回夏季大会）】\n開催地：韓国・ソウル\n期間：19...,0.374820
